In [5]:
from ImgPrep import file_names, read_img, reshape_array, split_img, display_img
import numpy as np

# list of files to read in
files = file_names('puzzle_scans/puzzle_1')
pictures = []
pic_arrays = []

for picture in files:
    
    # store picture as array
    pic = read_img(picture)

    # split each image into 20 pieces
    pieces = split_img(pic, 5, 4)
    pictures += pieces
    assert all([piece.shape for piece in pieces]), 'All arrays are not the same size'

    # split each piece into n by n parts
    n = 3

    # split each piece into quarters
    chopped = []
    for piece in pieces:
        chopped.append(split_img(piece, n, n))    
    assert all([part.shape for piece in chopped for part in piece]), \
    'All parts of all pieces are not the same size'

    # reshape arrays and find average RGB value of each part of each piece    
    avg_rgb = np.zeros((len(chopped), n**2, 3))
    for i,piece in enumerate(chopped):
        for ii,part in enumerate(piece):
                chopped[i][ii] = reshape_array(part)
                avg_rgb[i,ii,:] = np.mean(chopped[i][ii], axis = 0)
    
    pic_arrays.append(avg_rgb)

# combine all pictures into one large array for clustering
pic_arrays = np.vstack(pic_arrays)
assert pic_arrays.shape == (300, n**2, 3), \
f'Output array should be size (300,{n**2},3) but is actually size {pic_arrays.shape}'